In [ ]:
import pandas as pd
import numpy as np

In [ ]:
YEAR = 2021

In [ ]:
df = pd.read_csv(f'../dataset/summary_{YEAR}.csv', encoding='utf-8-sig')

def read_file(year):
    PATH = f'../dataset/imbank_customer_data/iMBANK_CUSTOMER_DATA_{year}(K-DigitalTraining).csv'
    df = pd.read_csv(PATH, encoding='euc-kr', usecols=['기준년월', '고객ID', '성별', '고객등급', '자택_시도', '수신_요구불예금', '수신_거치식예금', '수신_적립식예금', '대출금액'])
    df = df.sort_values(by='기준년월', ascending=True).reset_index(drop=True)
    return df

M_CS = read_file(YEAR)

# 고객 id 
filtered_data = df[df['연령대'].isin(['20대', '30대'])]['고객ID'].unique()

# 20, 30대의 고객 ID만 뽑음
M_filtered_data = M_CS[M_CS['고객ID'].isin(filtered_data)]

# 수신잔고, sum(요구불 ~ 적립식)
M_filtered_data['수신잔고'] = M_filtered_data.groupby(['기준년월','고객ID'])[['수신_요구불예금', '수신_거치식예금', '수신_적립식예금']].transform('sum').sum(axis=1)

# 지역구분 대구, 경상북도 => 1, 나머지 => 0
M_filtered_data['지역구분'] = np.where(M_filtered_data['자택_시도'].isin(['대구광역시', '경상북도']), 1, 0)                                                                

# 수신대비대출비율
M_filtered_data['수신대비대출비율'] = np.where(M_filtered_data['수신잔고'] != 0, M_filtered_data['대출금액'] / M_filtered_data['수신잔고'], 0)

result = M_filtered_data.merge(df[['고객ID', '연령대', '이탈여부']], on='고객ID', how='left')
result

In [ ]:
def read_file1(year):
    PATH = f'../dataset/imbank_card_data/iMBANK_CARD_DATA_{year}(K-DigitalTraining).csv'
    df = pd.read_csv(PATH, encoding='euc-kr', usecols=['거래년월','고객ID','승인건수','승인금액'])
    df = df.sort_values(by='거래년월', ascending=True).reset_index(drop=True)
    # 승인건수를 숫자로 변환 (필요한 경우 오류 처리)
    
    # 승인건수를 범주화
    df.loc[df['승인건수'] == '5번이하', '승인건수'] = 1
    df.loc[df['승인건수'] == '5번초과 10번이하', '승인건수'] = 2
    df.loc[df['승인건수'] == '10번초과 15번이하', '승인건수'] = 3
    df.loc[df['승인건수'] == '15번초과 20번이하', '승인건수'] = 4
    df.loc[df['승인건수'] == '20번 초과', '승인건수'] = 5

    card_date =df.groupby(['거래년월', '고객ID'])
    p = card_date['승인건수'].sum() / card_date['승인건수'].count()
    p = p.reset_index()

    new_card = pd.merge(new_card, p, on=['거래년월', '고객ID'], how='left')
    new_card.drop(['승인건수_x'], axis = 1, inplace=True)
    _sum = new_card.groupby(['거래년월', '고객ID'])['승인금액'].transform('sum')

    new_card = pd.merge(new_card, _sum, on=['거래년월', '고객ID'], how='left')
    new_card.columns = ['거래년월', '고객ID', '이탈여부', '승인건수', '월별승인금액합계']
    new_card.to_csv(f"../dataset/imbank_card_data/samp_card_{year}.csv", encoding='euc-kr', index=False)
    return df

C = read_file1(YEAR)